![](https://miro.medium.com/max/1079/1*G1-n0R-c0IQ4GMrAzDNySw.png)medium.com 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
nRowsRead = 1000 # specify 'None' if want to read whole file
df = pd.read_csv('../input/cusersmarildownloadsgermancsv/german.csv', delimiter=';', encoding = "ISO-8859-2", nrows = nRowsRead)
df.dataframeName = 'german.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')
df.head()

#Code by  Abhishek Singhal  https://www.kaggle.com/abhhii/pcaimplement/notebook

In [ ]:
data = df.copy(deep=True)
data = data.dropna(how='any')

#A small dataset where each rocord is a 3 dimensional vector

In [ ]:
smalldata = data[['Age_years', 'Account_Balance', 'Duration_of_Credit_monthly']]
smalldata.head()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

import matplotlib.pyplot as plt
fig = plt.figure()
ax = Axes3D(fig)

# data for 3d plot
ax.scatter(smalldata['Age_years'], smalldata['Account_Balance'], smalldata['Duration_of_Credit_monthly'])

In [ ]:
#Standardizing data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
std_data = scaler.fit_transform(smalldata)
std_data

In [ ]:
mean_vec = np.mean(std_data,axis=0)
cov_mat = (std_data - mean_vec).T.dot((std_data - mean_vec)) / (std_data.shape[0]-1)

#or we can directly use numpy library method
np.cov(std_data.T)

In [ ]:
#eigen decompostion of the covariance matrix
eig_vals, eig_vectors = np.linalg.eig(cov_mat)

print("Eigen values:", eig_vals)
print("Eigen vectors:", eig_vectors)

In [ ]:
 # correlation matrix
# in most datasets especially financial datasets eigendecomposition on covariance matrix and
# correlation matrix yields the same results since correlation matrix can be
# understood as the normalized covariance matrix
# so here is the eigendecompostion on correlation matrix
corr_mat = np.corrcoef(std_data.T)
eig_vals, eig_vectors = np.linalg.eig(corr_mat)
print("Eigen Values:", eig_vals)
print("Eigen vectors:", eig_vectors)

In [ ]:
# eigendecomposition of raw data based on correlation matri
corr_mat_raw = np.corrcoef(std_data.T)
eig_vals, eig_vecs = np.linalg.eig(corr_mat_raw)
print('Eigenvals are:', eig_vals)
print('eigenvectors are:', eig_vecs)

In [ ]:
# while eigendecompostion of covariance matrix is more intutive, most implementations use singular value decompostion to increase computational efficiency
u,s,v = np.linalg.svd(std_data.T)
u

#Selecting Principal Components

In [ ]:
# make a list of (eigenvalue, eigenvectors) tuples
eig_pairs = [(eig_vals[i], eig_vecs[i]) for i in range(len(eig_vals))]
# eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:i]) for i in range(len(eig_vals))]

# print("Eig pairs are\n",eig_pairs )
# sort the eigenvalue, eigenvector tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

for i in eig_pairs:
    print(i[0])

In [ ]:
#Estimating the importance of each feature vector using explined variance

import matplotlib.pyplot as plt
tot = sum(eig_vals)
var_exp = [(i/tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

x_coordinates = ('PC1', 'PC2', 'PC3')
y_pos = np.arange(len(x_coordinates))

plt.bar(y_pos, var_exp, align='center')
plt.ylabel('Explained var in %')
plt.xticks(y_pos, x_coordinates)
plt.plot(cum_var_exp, 'r')
plt.show()

This plot shows that around 58% variance can be explained by first component and around 30% variance is explained by second component. Thus they both cover around 88% of the variance and third component can be dropped without losing too much information.

In [ ]:
eig_pairs

In [ ]:
# making the new projection matrix 
pro_mat = np.hstack((eig_pairs[0][1].reshape(3,1),
                   eig_pairs[1][1].reshape(3,1)))

print('Projection Matrix :\n', pro_mat)

In [ ]:
pro_mat.shape

In [ ]:
std_data

In [ ]:
new_mat = std_data.dot(pro_mat)
new_mat

In [ ]:
X = [new_mat[i][0] for i in range(len(new_mat))]
y = [new_mat[i][1] for i in range(len(new_mat))]
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.scatter(X, y)

In [ ]:
#Code by Olga Belitskaya https://www.kaggle.com/olgabelitskaya/sequential-data/comments
from IPython.display import display,HTML
c1,c2,f1,f2,fs1,fs2=\
'#eb3434','#eb3446','Akronim','Smokum',30,15
def dhtml(string,fontcolor=c1,font=f1,fontsize=fs1):
    display(HTML("""<style>
    @import 'https://fonts.googleapis.com/css?family="""\
    +font+"""&effect=3d-float';</style>
    <h1 class='font-effect-3d-float' style='font-family:"""+\
    font+"""; color:"""+fontcolor+"""; font-size:"""+\
    str(fontsize)+"""px;'>%s</h1>"""%string))
    
    
dhtml('Thanks to Abhishek Singhal for all the code' )